In [9]:
from google.cloud import bigquery
from google.oauth2 import service_account
import requests
import json
from dotenv import load_dotenv
import os

load_dotenv()

True

In [10]:
credentials = service_account.Credentials.from_service_account_file(
    'creds.json',
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id,
)

In [11]:
query = client.query("""
   SELECT *
   FROM ddp_intermediate.students_activities_agg
""")
bigquery_results = query.result()

In [13]:
# API_SECRET = 'ishan'
URI = 'https://staging-tap-lms.openbrackets.in/'
# authenticate to frappe as admin
# headers = {"Authorization": f"token api_key:{API_SECRET}"}
payload = {'usr': os.getenv('FRAPPE_ADMIN_USERNAME'), 'pwd': os.getenv('FRAPPE_ADMIN_PASSWORD')}
res = requests.post(f"{URI}/api/method/login", data=payload)
cookies = res.cookies
# do subsequent request with this cookies
cookies_dict = cookies.get_dict()

In [14]:
res = requests.get(f"{URI}/api/resource/Student", cookies=cookies_dict, params={'fields': json.dumps(['name', 'phone'])})
students = res.json()['data']

In [15]:
students

[{'name': 'fa5b0e3689', 'phone': '917666491404'},
 {'name': 'c1fb9b684b', 'phone': '8551065721'},
 {'name': '8790c880a1', 'phone': '7058947337'},
 {'name': '7aa716c6e7', 'phone': '9420081117'},
 {'name': 'b69e588cda', 'phone': '8624097377'},
 {'name': '40b00cdabe', 'phone': '8485811763'},
 {'name': '04eb27b167', 'phone': '9013183473'},
 {'name': '12b76bdf7f', 'phone': '9911815753'},
 {'name': '8008e29f4d', 'phone': '9628705100'},
 {'name': 'a68a082f65', 'phone': '8587017713'},
 {'name': 'a80852471b', 'phone': '9871218118'},
 {'name': '170ecedbd0', 'phone': '8586948631'},
 {'name': '08fea7dac0', 'phone': '8375840766'},
 {'name': '298ec22e79', 'phone': '8742969921'},
 {'name': '31aa127580', 'phone': '9310954922'},
 {'name': '91baf1306a', 'phone': '7331171079'},
 {'name': 'a606a22048', 'phone': '9618518944'},
 {'name': 'a09d0b50b1', 'phone': '9515650980'},
 {'name': '8c739b38e8', 'phone': '7093723697'},
 {'name': '11fe1d13a9', 'phone': '7989713170'}]

In [16]:
prepare_data_to_put = []
for row in bigquery_results:
    d = {}
    d['access_rate'] = row.access_rate
    d['submission_rate'] = row.submission_rate

    for student in students:
        if student['phone'] == row.phone:
            d['name'] = student['name']
            prepare_data_to_put.append(d)
            break

In [43]:
# upload metrics to frappe

for data in prepare_data_to_put:
    res = requests.put(f"{URI}/api/resource/Student/{data['name']}", \
                        cookies=cookies_dict, \
                        data={"submission_rate": data['submission_rate'], "access_rate": data['access_rate']})
    if(res.status_code != 200):
        print('not ok')
        break

In [17]:
client.close()